In [1]:
library(Seurat)
library(tidyverse)
library(patchwork)
options(tibble.print_max = 200, width=200)

Attaching SeuratObject

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
xenium.obj = qs::qread('_targets/objects/obj_merged')
xenium.obj

An object of class Seurat 
400 features across 119775 samples within 4 assays 
Active assay: Xenium (100 features, 0 variable features)
 3 other assays present: BlankCodeword, ControlCodeword, ControlProbe
 8 spatial fields of view present: fov fov.1 fov.2 fov.3 fov.4 fov.5 fov.6 fov.7

In [ ]:
# remove cells with 0 counts
xenium.obj <- subset(xenium.obj, subset = nCount_Xenium > 0)
# Create a vector of cells to keep
keep_cells = xenium.obj %>%
`[[` %>%
rownames_to_column %>%
filter(rowname != "0") %>%
pull(rowname)
# Subset the Seurat object to drop the cells
xenium.obj <- subset(xenium.obj, cells=keep_cells)
xenium.obj

In [ ]:
summary_tibble <- xenium.obj %>% 
`[[` %>%
#   group_by(seurat_clusters) %>%
  summarise(
    n = n(),
    min = min(nCount_Xenium),
    max = max(nCount_Xenium),
    mean = mean(nCount_Xenium),
    stdev = sd(nCount_Xenium),
    quantiles = list(quantile(nCount_Xenium, probs = c(seq(0, 0.9, by=0.1), seq(0.91, 1, by= 0.01)), names = FALSE))
  ) %>%
  unnest_wider(quantiles, names_sep = "") %>%
  rename_with(~paste0("Q", c(seq(0, 0.9, by=0.1), seq(0.91, 1, by= 0.01))), starts_with("quantiles"))

summary_tibble %>% print


In [ ]:
summary_tibble <- xenium.obj %>% 
`[[` %>%
#   group_by(seurat_clusters) %>%
 filter(!is.na(nucleus_area)) %>%
  summarise(
    n = n(),
    min = min(nucleus_area),
    max = max(nucleus_area),
    mean = mean(nucleus_area),
    stdev = sd(nucleus_area),
    quantiles = list(quantile(nucleus_area, probs = c(seq(0, 0.10, by=0.01), seq(0.2, 0.9, by=0.1), seq(0.91, 1, by= 0.01)), names = FALSE))
  ) %>%
  unnest_wider(quantiles, names_sep = "") %>%
  rename_with(~paste0("Q", c(seq(0, 0.10, by=0.01), seq(0.2, 0.9, by=0.1), seq(0.91, 1, by= 0.01))), starts_with("quantiles")) %>%
t

summary_tibble 


In [ ]:
summary_tibble <- xenium.obj %>% 
`[[` %>%
#   group_by(seurat_clusters) %>%
 filter(!is.na(nFeature_Xenium)) %>%
  summarise(
    n = n(),
    min = min(nFeature_Xenium),
    max = max(nFeature_Xenium),
    mean = mean(nFeature_Xenium),
    stdev = sd(nFeature_Xenium),
    quantiles = list(quantile(nFeature_Xenium, probs = c(seq(0, 0.10, by=0.01), seq(0.2, 0.9, by=0.1), seq(0.91, 1, by= 0.01)), names = FALSE))
  ) %>%
  unnest_wider(quantiles, names_sep = "") %>%
  rename_with(~paste0("Q", c(seq(0, 0.10, by=0.01), seq(0.2, 0.9, by=0.1), seq(0.91, 1, by= 0.01))), starts_with("quantiles")) %>%
t

summary_tibble 


In [ ]:
summary_tibble <- xenium.obj %>% 
`[[` %>%
#   group_by(seurat_clusters) %>%
  summarise(
    n = n(),
    min = min(cell_area, na.rm = TRUE),
    max = max(cell_area, na.rm = TRUE),
    mean = mean(cell_area, na.rm = TRUE),
    stdev = sd(cell_area, na.rm = TRUE),
    quantiles = list(quantile(cell_area, probs = c(seq(0, 0.9, by=0.1), seq(0.91, 1, by= 0.01)), names = FALSE, na.rm = TRUE))
  ) %>%
  unnest_wider(quantiles, names_sep = "") %>%
  rename_with(~paste0("Q", c(seq(0, 0.9, by=0.1), seq(0.91, 1, by= 0.01))), starts_with("quantiles"))


summary_tibble %>% print


In [ ]:
xenium.obj %>% `[[` %>% colnames

In [3]:
keep_cells = xenium.obj %>% `[[` %>%
filter(cell_area < 7000) %>%
filter(cell_area > 360) %>%
filter(avg_confidence >= 0.97) %>%
filter(nCount_Xenium >= 10) %>%
filter(nCount_Xenium <= 600) %>%
filter(nFeature_Xenium >= 10) %>%
filter(nFeature_Xenium <= 70) %>%
rownames

length(keep_cells)

[1] 78054

In [4]:
xenium.obj <- subset(xenium.obj, cells=keep_cells)
xenium.obj

An object of class Seurat 
400 features across 78054 samples within 4 assays 
Active assay: Xenium (100 features, 0 variable features)
 3 other assays present: BlankCodeword, ControlCodeword, ControlProbe
 8 spatial fields of view present: fov fov.1 fov.2 fov.3 fov.4 fov.5 fov.6 fov.7

In [5]:
xenium.obj = Seurat::SCTransform(xenium.obj,
                        assay='Xenium',
                        method="glmGamPoi",
                        vst.flavor="v2",
                        verbose=TRUE)
xenium.obj

vst.flavor='v2' set, setting model to use fixed slope and exclude poisson genes.

Calculating cell attributes from input UMI matrix: log_umi

Total Step 1 genes: 99

Total overdispersed genes: 99

Excluding 0 genes from Step 1 because they are not overdispersed.

Variance stabilizing transformation of count matrix of size 100 by 78054

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 99 genes, 5000 cells



  |==============================================================================================================================================================================================| 100%


Setting estimate of  0 genes to inf as theta_mm/theta_mle < 1e-3

# of step1 poisson genes (variance < mean): 0

# of low mean genes (mean < 0.001): 1

Total # of Step1 poisson genes (theta=Inf; variance < mean): 0

Total # of poisson genes (theta=Inf; variance < mean): 1

Calling offset model for all 1 poisson genes

Found 1 outliers - those will be ignored in fitting/regularization step


Ignoring theta inf genes

Replacing fit params for 1 poisson genes by theta=Inf

Setting min_variance based on median UMI:  0.16

Second step: Get residuals using fitted parameters for 100 genes



  |==============================================================================================================================================================================================| 100%


Computing corrected count matrix for 100 genes



  |==============================================================================================================================================================================================| 100%


Calculating gene attributes

Wall clock passed: Time difference of 15.32895 secs

Determine variable features

Place corrected count matrix in counts slot

Centering data matrix

Set default assay to SCT

Warning message:
“Invalid name supplied, making object name syntactically valid. New object name is Seurat..SCTransform.Xenium; see ?make.names for more details on syntax validity”


An object of class Seurat 
500 features across 78054 samples within 5 assays 
Active assay: SCT (100 features, 100 variable features)
 4 other assays present: Xenium, BlankCodeword, ControlCodeword, ControlProbe
 8 spatial fields of view present: fov fov.1 fov.2 fov.3 fov.4 fov.5 fov.6 fov.7

In [7]:
qs::qsave(xenium.obj, 'xenium.obj.qs')

In [61]:
obj_fgf1 = qs::qread('/projects/petar/fgf1/full_obj_sct.qs')
obj_fgf1

An object of class Seurat 
59498 features across 144643 samples within 3 assays 
Active assay: SCT (27177 features, 3000 variable features)
 2 other assays present: RNA, integrated
 2 dimensional reductions calculated: pca, umap

In [62]:
xenium_genes = xenium.obj@assays$Xenium@meta.features %>% rownames
ref_genes = obj_fgf1@assays$SCT@meta.features %>% rownames
xenium_genes = xenium_genes %>% intersect(ref_genes)
xenium_genes = xenium_genes %>% tibble %>% filter(. != 'Lmx1a') %>% pull

length(xenium_genes)
xenium_genes

[1] 98

[1] "Sorcs1"   "Lingo2"   "Cntn4"    "Tenm2"    "Sox5"     "Ptprd"    "Robo1"    "Plcl1"    "Oxr1"     "Plxdc2"   "Plp1"     "Kirrel3"  "Mgat4c"   "Tenm3"    "Slit2"    "Nrg3"     "Hdac9"   
[18] "Col25a1"  "Ncam2"    "Rbfox1"   "Fgf13"    "Pde10a"   "Deptor"   "Alcam"    "Gria1"    "Grid2"    "Grm7"     "Agrp"     "Naaladl2" "Ntm"      "Gpc5"     "Hs3st4"   "Lef1"     "Erbb4"   
[35] "Kctd16"   "Zfhx3"    "Gtdc1"    "Pdgfra"   "Nrg1"     "Macrod2"  "Gpc6"     "Pcdh11x"  "Kcnip4"   "Brinp3"   "Cadm2"    "Pde4b"    "Fhit"     "Rgs6"     "Prkg1"    "Dlg2"     "Ptprk"   
[52] "Luzp2"    "Ptprt"    "Nxph1"    "Kcnq3"    "Lrmda"    "Dcc"      "Cntn5"    "Adarb2"   "Gabrg3"   "Rmst"     "Lrp1b"    "Trpm3"    "P3h2"     "Prr16"    "Prkca"    "Nrxn3"    "Aqp4"    
[69] "Ctnna2"   "Inpp4b"   "Sgcz"     "Fat3"     "Plcb1"    "Cacna2d3" "Bmp4"     "Tox"      "Car10"    "Nkain2"   "Galntl6"  "Slc7a11"  "Pcdh9"    "Slc1a2"   "Pdzrn3"   "Rax"      "Pomc"    
[86] "Zfp804b"  "Wwox"     "Pcdh7"    "Grm8"     "Rtl4"     "Il1rapl2" "Sgcd"     "Egfem1"   "Arhgap6"  "Immp2l"   "Cfap299"  "Htr3b"    "Csmd1"

In [63]:
source('../00_preprocessing/preprocessing.R')

In [64]:
obob5_cells = obj_fgf1 %>%
`[[` %>%
filter(strain == 'obob') %>%
filter(time == 'Day5') %>%
rownames

In [ ]:
obj_fgf1 = subset(obj_fgf1, cells=obob5_cells) %>%
sc_transform_fgf1 %>%
Seurat::RunUMAP(assay='SCT', slot='counts', features = xenium_genes, return.model = TRUE)
obj_fgf1

vst.flavor='v2' set, setting model to use fixed slope and exclude poisson genes.

Calculating cell attributes from input UMI matrix: log_umi

Total Step 1 genes: 22228

Total overdispersed genes: 21740

Excluding 488 genes from Step 1 because they are not overdispersed.

Variance stabilizing transformation of count matrix of size 26292 by 58235

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 2000 genes, 5000 cells



  |==============================================================================================================================================================================================| 100%


Setting estimate of  264 genes to inf as theta_mm/theta_mle < 1e-3

# of step1 poisson genes (variance < mean): 0

# of low mean genes (mean < 0.001): 4165

Total # of Step1 poisson genes (theta=Inf; variance < mean): 296

Total # of poisson genes (theta=Inf; variance < mean): 4535

Calling offset model for all 4535 poisson genes

Found 375 outliers - those will be ignored in fitting/regularization step


Ignoring theta inf genes

Replacing fit params for 4535 poisson genes by theta=Inf



In [ ]:
obj_fgf1 %>% qs::qsave('obj_fgf1_obob5v5.qs')

# All Xe genes integ

In [51]:
xenium.obj = qs::qread('xenium.obj.qs')
xenium.obj = RunUMAP(xenium.obj, assay='SCT', slot='counts', features = xenium_genes, return.model = TRUE)
qs::qsave(xenium.obj, 'xenium.obj.qs')
xenium.obj

UMAP will return its model

14:11:39 UMAP embedding parameters a = 0.9922 b = 1.112

14:11:39 Read 78054 rows and found 98 numeric columns

14:11:39 Using Annoy for neighbor search, n_neighbors = 30

14:11:39 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

14:11:59 Writing NN index file to temp file /scratch/nmq407/R_tmp//Rtmp1BxZIS/file8ce73e6d9e3c

14:11:59 Searching Annoy index using 1 thread, search_k = 3000

14:13:02 Annoy recall = 98.96%

14:13:04 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

14:13:10 Initializing from normalized Laplacian + noise (using irlba)

14:13:28 Commencing optimization for 200 epochs, with 3867446 positive edges

14:15:20 Optimization finished



An object of class Seurat 
500 features across 78054 samples within 5 assays 
Active assay: SCT (100 features, 100 variable features)
 4 other assays present: Xenium, BlankCodeword, ControlCodeword, ControlProbe
 1 dimensional reduction calculated: umap
 8 spatial fields of view present: fov fov.1 fov.2 fov.3 fov.4 fov.5 fov.6 fov.7

In [68]:
xenium.obj

An object of class Seurat 
500 features across 78054 samples within 5 assays 
Active assay: SCT (100 features, 100 variable features)
 4 other assays present: Xenium, BlankCodeword, ControlCodeword, ControlProbe
 1 dimensional reduction calculated: umap
 8 spatial fields of view present: fov fov.1 fov.2 fov.3 fov.4 fov.5 fov.6 fov.7

In [10]:
obj_fgf1 = qs::qread('obj_fgf1_obob5v5.qs')
obj_fgf1

An object of class Seurat 
58613 features across 58235 samples within 3 assays 
Active assay: SCT (26292 features, 3000 variable features)
 2 other assays present: RNA, integrated
 2 dimensional reductions calculated: pca, umap

In [69]:
xenium_genes = xenium.obj@assays$SCT@meta.features %>% rownames
ref_genes = obj_fgf1@assays$SCT@meta.features %>% rownames
xenium_genes = xenium_genes %>% intersect(ref_genes)
xenium_genes = xenium_genes %>% tibble %>% filter(. != 'Lmx1a') %>% pull

length(xenium_genes)
xenium_genes

[1] 98

[1] "Sorcs1"   "Lingo2"   "Cntn4"    "Tenm2"    "Sox5"     "Ptprd"    "Robo1"    "Plcl1"    "Oxr1"     "Plxdc2"   "Plp1"     "Kirrel3"  "Mgat4c"   "Tenm3"    "Slit2"    "Nrg3"     "Hdac9"   
[18] "Col25a1"  "Ncam2"    "Rbfox1"   "Fgf13"    "Pde10a"   "Deptor"   "Alcam"    "Gria1"    "Grid2"    "Grm7"     "Agrp"     "Naaladl2" "Ntm"      "Gpc5"     "Hs3st4"   "Lef1"     "Erbb4"   
[35] "Kctd16"   "Zfhx3"    "Gtdc1"    "Pdgfra"   "Nrg1"     "Macrod2"  "Gpc6"     "Pcdh11x"  "Kcnip4"   "Brinp3"   "Cadm2"    "Pde4b"    "Fhit"     "Rgs6"     "Prkg1"    "Dlg2"     "Ptprk"   
[52] "Luzp2"    "Ptprt"    "Nxph1"    "Kcnq3"    "Lrmda"    "Dcc"      "Cntn5"    "Adarb2"   "Gabrg3"   "Rmst"     "Lrp1b"    "Trpm3"    "P3h2"     "Prr16"    "Prkca"    "Nrxn3"    "Aqp4"    
[69] "Ctnna2"   "Inpp4b"   "Sgcz"     "Fat3"     "Plcb1"    "Cacna2d3" "Bmp4"     "Tox"      "Car10"    "Nkain2"   "Galntl6"  "Slc7a11"  "Pcdh9"    "Slc1a2"   "Pdzrn3"   "Rax"      "Pomc"    
[86] "Zfp804b"  "Wwox"     "Pcdh7"    "Grm8"     "Rtl4"     "Il1rapl2" "Sgcd"     "Egfem1"   "Arhgap6"  "Immp2l"   "Cfap299"  "Htr3b"    "Csmd1"

In [70]:
cat(xenium_genes)

Sorcs1 Lingo2 Cntn4 Tenm2 Sox5 Ptprd Robo1 Plcl1 Oxr1 Plxdc2 Plp1 Kirrel3 Mgat4c Tenm3 Slit2 Nrg3 Hdac9 Col25a1 Ncam2 Rbfox1 Fgf13 Pde10a Deptor Alcam Gria1 Grid2 Grm7 Agrp Naaladl2 Ntm Gpc5 Hs3st4 Lef1 Erbb4 Kctd16 Zfhx3 Gtdc1 Pdgfra Nrg1 Macrod2 Gpc6 Pcdh11x Kcnip4 Brinp3 Cadm2 Pde4b Fhit Rgs6 Prkg1 Dlg2 Ptprk Luzp2 Ptprt Nxph1 Kcnq3 Lrmda Dcc Cntn5 Adarb2 Gabrg3 Rmst Lrp1b Trpm3 P3h2 Prr16 Prkca Nrxn3 Aqp4 Ctnna2 Inpp4b Sgcz Fat3 Plcb1 Cacna2d3 Bmp4 Tox Car10 Nkain2 Galntl6 Slc7a11 Pcdh9 Slc1a2 Pdzrn3 Rax Pomc Zfp804b Wwox Pcdh7 Grm8 Rtl4 Il1rapl2 Sgcd Egfem1 Arhgap6 Immp2l Cfap299 Htr3b Csmd1

In [74]:
# anchors = FindIntegrationAnchors(obj_list, assay=c('SCT', 'SCT'), anchor.features=xenium_genes)
anchors <- FindTransferAnchors(reference = obj_fgf1, query = xenium.obj, features = xenium_genes,  
                               normalization.method = 'SCT',
                               reduction = 'pcaproject',
                               query.assay = 'SCT',
                               reference.assay = 'SCT',
                               recompute.residuals = F,
                               reference.reduction='pca',
                               verbose=TRUE)

Warning message:
“Adding image data that isn't associated with any assay present”
Warning message:
“Adding image data that isn't associated with any assay present”
Warning message:
“Adding image data that isn't associated with any assay present”
Warning message:
“Adding image data that isn't associated with any assay present”
Warning message:
“Adding image data that isn't associated with any assay present”
Warning message:
“Adding image data that isn't associated with any assay present”
Warning message:
“Adding image data that isn't associated with any assay present”
Warning message:
“Adding image data that isn't associated with any assay present”
Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 21923 anchors



In [75]:
xenium.obj_unimapped <- MapQuery(anchorset = anchors, reference = obj_fgf1, query = xenium.obj,
                                 refdata = list(labels = "labels"), reduction.model = "umap")

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”

Integrating dataset 2 with reference dataset

Finding integration vectors

Integrating data

Computing nearest neighbors

Running UMAP projection

15:50:58 Read 78054 rows

15:50:58 Processing block 1 of 1

15:50:58 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

15:51:00 Initializing by weighted average of neighbor coordinates using 1 thread

15:51:00 Commencing optimization for 67 epochs, with 2341619 positive edges

15:51:24 Finished



In [76]:
qs::qsave(xenium.obj_unimapped, '230502_xenium_sct_unimapped.qs')

In [77]:
# anchors = FindIntegrationAnchors(obj_list, assay=c('SCT', 'SCT'), anchor.features=xenium_genes)
anchors_cca <- FindTransferAnchors(reference = obj_fgf1, query = xenium.obj, features = xenium_genes,  
                               normalization.method = 'SCT',
                               reduction = 'cca',
                               query.assay = 'SCT',
                               reference.assay = 'SCT',
                               recompute.residuals = F,
                               verbose=TRUE)

Warning message:
“Adding image data that isn't associated with any assay present”
Warning message:
“Adding image data that isn't associated with any assay present”
Warning message:
“Adding image data that isn't associated with any assay present”
Warning message:
“Adding image data that isn't associated with any assay present”
Warning message:
“Adding image data that isn't associated with any assay present”
Warning message:
“Adding image data that isn't associated with any assay present”
Warning message:
“Adding image data that isn't associated with any assay present”
Warning message:
“Adding image data that isn't associated with any assay present”
Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 77147 anchors



In [78]:
xenium.obj_unimapped_cca <- MapQuery(anchorset = anchors_cca, reference = obj_fgf1, query = xenium.obj,
                                 refdata = list(labels = "labels"), reduction.model = "umap")

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Computing nearest neighbors

Running UMAP projection

16:04:27 Read 78054 rows

16:04:27 Processing block 1 of 1

16:04:27 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

16:04:29 Initializing by weighted average of neighbor coordinates using 1 thread

16:04:29 Commencing optimization for 67 epochs, with 2341620 positive edges

16:05:18 Finished



In [79]:
qs::qsave(xenium.obj_unimapped_cca, '230502_xenium_sct_unimapped_cca.qs')

In [81]:
xenium.obj

An object of class Seurat 
500 features across 78054 samples within 5 assays 
Active assay: SCT (100 features, 100 variable features)
 4 other assays present: Xenium, BlankCodeword, ControlCodeword, ControlProbe
 1 dimensional reduction calculated: umap
 8 spatial fields of view present: fov fov.1 fov.2 fov.3 fov.4 fov.5 fov.6 fov.7

In [80]:
# anchors = FindIntegrationAnchors(obj_list, assay=c('SCT', 'SCT'), anchor.features=xenium_genes)
anchors <- FindTransferAnchors(reference = obj_fgf1, query = xenium.obj, features = xenium_genes,  
                               normalization.method = 'SCT',
                               reduction = 'pcaproject',
                               query.assay = 'Xenium',
                               reference.assay = 'SCT',
                               reference.reduction = "pca", 
                               recompute.residuals = F,
                               verbose=TRUE)

Warning message:
“98 features of the features specified were not present in both the reference query assays. 
Continuing with remaining 0 features.”


ERROR: Error: No features to use in finding transfer anchors. To troubleshoot, try explicitly providing features to the features parameter and ensure that they are present in both reference and query assays.


In [ ]:
xenium.obj_unimapped <- MapQuery(anchorset = anchors, reference = obj_fgf1, query = xenium.obj,
                                 refdata = list(labels = "labels"), reference.reduction = "pca", reduction.model = "umap")

In [ ]:
qs::qsave(xenium.obj_unimapped, '230502_xenium_unimapped_fgf1XEgenesUMAP.qs')

In [ ]:
# anchors = FindIntegrationAnchors(obj_list, assay=c('SCT', 'SCT'), anchor.features=xenium_genes)
anchors_cca <- FindTransferAnchors(reference = obj_fgf1, query = xenium.obj, features = xenium_genes,  
                               normalization.method = 'SCT',
                               reduction = 'cca',
                               query.assay = 'Xenium',
                               reference.assay = 'SCT',
                               recompute.residuals = F,
                               verbose=TRUE)

In [ ]:
xenium.obj_unimapped_cca <- MapQuery(anchorset = anchors_cca, reference = obj_fgf1, query = xenium.obj,
                                 refdata = list(labels = "labels"), reduction.model = "umap")

In [ ]:
qs::qsave(xenium.obj_unimapped_cca, '230502_xenium_unimapped_cca_fgf1XEgenesUMAP.qs')